In [1]:
from botorch.models import SingleTaskGP, ModelListGP
import torch
import time
import psutil
import os
from functools import wraps
from contextlib import redirect_stdout
import logging
import json


from botorch.fit import fit_gpytorch_mll
from botorch.models.transforms.outcome import Standardize
from gpytorch.mlls import SumMarginalLogLikelihood
from gpytorch.kernels import (
    ScaleKernel,
    MaternKernel,
    RBFKernel,
)


# --- Logging 配置 ---
logger = logging.getLogger("BO_Monitor")
logger.setLevel(logging.INFO)
if not logger.handlers:
    ch = logging.StreamHandler()
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    ch.setFormatter(formatter)
    logger.addHandler(ch)


# ============================================================
# Decorator
# ============================================================

def monitor(runtime=True, memory=True, condition=False):
    def decorator(func):
        @wraps(func)
        def wrapper(self, *args, **kwargs):
            if not getattr(self, "debug", False):
                return func(self, *args, **kwargs)

            # 確保類別中有存儲紀錄的容器
            if not hasattr(self, 'performance_history'):
                self.performance_history = []

            # 紀錄開始資訊
            start_time = time.perf_counter()
            process = psutil.Process(os.getpid())
            
            # 執行函數
            result = func(self, *args, **kwargs)
            
            # 計算指標
            end_time = time.perf_counter()
            elapsed = end_time - start_time
            cpu_mem = process.memory_info().rss / 1024**2
            
            entry = {
                "iteration": getattr(self, "current_iter", "N/A"), # 追蹤當前疊代次數
                "function": func.__name__,
                "runtime_sec": round(elapsed, 4),
                "cpu_memory_mb": round(cpu_mem, 2),
                "timestamp": time.strftime("%Y-%m-%d %H:%M:%S")
            }

            if torch.cuda.is_available():
                entry["gpu_alloc_mb"] = round(torch.cuda.memory_allocated() / 1024**2, 2)

            # 如果需要紀錄條件數 (僅針對 fit 後的模型)
            if condition and getattr(self, "model", None) is not None:
                cond_list = []
                for i, m in enumerate(self.model.models):
                    with torch.no_grad():
                        K = m.covar_module(m.train_inputs[0]).evaluate()
                        eigvals = torch.linalg.eigvalsh(K)
                        cond = (eigvals.max() / eigvals.min()).item()
                        cond_list.append(cond)
                entry["condition_numbers"] = cond_list

            # 儲存到物件歷史紀錄
            self.performance_history.append(entry)
            
            # 同時印出日誌供參考
            logger.info(f"Finished {func.__name__} - {entry}")
            
            return result
        return wrapper
    return decorator




class BaselineGPEmulator:
    """
    Baseline Multi-Objective GP Emulator
    =====================================

    This class implements a multi-objective Gaussian Process (GP)
    surrogate model using independent SingleTaskGP models.

    Mathematical Form:

        y_k(z) = m_k(z) + g_k(z)

        m_k(z) : constant mean (default in SingleTaskGP)
        g_k(z) ~ GP(0, K_{θ_k})

    Each objective is modeled with an independent GP.

    Example:
        >>> import torch
        >>> from baseline_gp_emulator import BaselineGPEmulator
        >>>
        >>> # Generate training data
        >>> train_X = torch.rand(20, 3, dtype=torch.double)
        >>> train_Y = torch.rand(20, 2, dtype=torch.double)
        >>>
        >>> # Create emulator
        >>>
        >>> # method1 : RBF is defult
        >>> emulator = BaselineGPEmulator()  #defult = RBF
        >>>
        >>> # method2 : matern
        >>> emulator = BaselineGPEmulator(
        ...     kernel="matern"
        ... )
        >>>
        >>> # Fit model
        >>> model = emulator.fit(train_X, train_Y)
        >>>
        >>> # Make predictions
        >>> test_X = torch.rand(5, 3, dtype=torch.double)
        >>> mean, var = emulator.predict(test_X)
        >>>
        >>> mean.shape
        torch.Size([5, 2])
        >>> var.shape
        torch.Size([5, 2])
    """

    def __init__(
        self,
        device  = torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        dtype   = torch.double,
        kernel    = "rbf",      # "rbf" or "matern"
        use_ard   = False,       # Whether to use Automatic Relevance Determination
        matern_nu = 2.5,         # Smoothness parameter for Matern kernel
        debug=False,   # Debug switch
        
    
    ):
        self.device = device
        self.dtype = dtype
        self.model = None
        self.mll = None

        self.kernel = kernel
        self.use_ard = use_ard
        self.matern_nu = matern_nu
        self.debug = debug
        self.performance_history = []
        self.current_iter = 0


    def _build_kernel(self, input_dim):

        """
        Construct covariance kernel based on configuration.

        If ARD is enabled, each input dimension has its own
        independent lengthscale parameter.
        """


        ard_dims = input_dim if self.use_ard else None

        if self.kernel == "matern":
            base_kernel = MaternKernel(
                nu=self.matern_nu,
                ard_num_dims=ard_dims,
            )
        else:  # default: RBF
            base_kernel = RBFKernel(
                ard_num_dims=ard_dims,
            )

        # ScaleKernel allows the model to learn an output scale parameter
        return ScaleKernel(base_kernel)


    @monitor(runtime=True, memory=True, condition=True)
    def fit(self, train_x: torch.Tensor, train_obj: torch.Tensor):
        """
        Fit the multi-objective GP surrogate model.

        Parameters
        ----------
        train_x : torch.Tensor
            Training inputs of shape (N, d)

        train_obj : torch.Tensor
            Training objectives of shape (N, k)

        Returns
        -------
        ModelListGP
            A ModelListGP containing independent SingleTaskGP models
            for each objective.
        """

        # If single objective (N,), convert to (N, 1)
        if train_obj.ndim == 1:
            train_obj = train_obj.unsqueeze(-1)

        input_dim = train_x.shape[-1]
        num_objectives = train_obj.shape[-1]

        models = []
        
        # Build one independent GP per objective
        for i in range(num_objectives):
            covar_module = self._build_kernel(input_dim)

            train_y = train_obj[..., i:i + 1]  # (N, 1)

            gp = SingleTaskGP(
                train_X=train_x,
                train_Y=train_y,
                outcome_transform=Standardize(m=1),
                covar_module=covar_module,
            )
            models.append(gp)

        # Combine independent GPs into ModelListGP
        self.model = ModelListGP(*models)

        # Define marginal log likelihood for multi-model case
        self.mll = SumMarginalLogLikelihood(
            self.model.likelihood, self.model
        )
        with open(os.devnull, 'w') as f:
            with redirect_stdout(f):
                fit_gpytorch_mll(self.mll)


        # Maximize marginal log likelihood
        fit_gpytorch_mll(self.mll)

        return self.model


    def save_performance_to_json(self, folder_path, filename="performance_report.json"):
        """
        將監控紀錄儲存至指定資料夾路徑。
        """
        # 1. 確保資料夾存在，若不存在則自動建立
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
            logger.info(f"Created directory: {folder_path}")

        # 2. 合併完整路徑
        full_path = os.path.join(folder_path, filename)

        # 3. 寫入 JSON
        try:
            with open(full_path, 'w', encoding='utf-8') as f:
                json.dump(self.performance_history, f, indent=4, ensure_ascii=False)
            logger.info(f"Successfully saved performance metrics to: {full_path}")
        except Exception as e:
            logger.error(f"Failed to save JSON: {str(e)}")


    @torch.no_grad()
    def predict(self, X ):
        """
        Predict using the trained GP surrogate.

        Parameters
        ----------
        X : torch.Tensor or array-like
            Input locations of shape (n, d)

        Returns
        -------
        mean : torch.Tensor
            Posterior predictive mean of shape (n, k)

        var : torch.Tensor
            Posterior predictive variance of shape (n, k)
        """

        if self.model is None:
            raise RuntimeError("Model has not been fitted yet.")
        
        X = torch.as_tensor(X, dtype=self.dtype, device=self.device)
        posterior = self.model.posterior(X)
        mean = posterior.mean    # (n, k)
        var = posterior.variance # (n, k)

        return mean, var






In [2]:
from botorch.models import SingleTaskGP, ModelListGP
from botorch.fit import fit_gpytorch_mll
from botorch.models.transforms.outcome import Standardize
from gpytorch.mlls import SumMarginalLogLikelihood
from gpytorch.means import Mean
from gpytorch.kernels import ScaleKernel, MaternKernel, RBFKernel

import torch
import itertools
import time
import psutil
import os
import logging
import json
from functools import wraps
from contextlib import redirect_stdout



# ============================================================
# Optimized Mean Function: IDXSFastScheffeMean 
# ============================================================
class IdxsFastScheffeMean(Mean):
    """
    High-performance Scheffé polynomial mean function.

    Mathematical form:

        m(z) =
            Σ β_i z_i
          + Σ β_ij z_i z_j
          + Σ β_ijk z_i z_j z_k
          + Σ β_ijkl z_i z_j z_k z_l

    depending on order.

    Design goals:
    - No torch.pow
    - No mask matrix
    - No large broadcast tensor
    - No Python inner loops in forward
    - Optimized for optimize_acqf gradient loops
    """

    def __init__(self, input_dim, order=2):

        """
        Parameters
        ----------
        input_dim : int
            Number of mixture components (dimension d)

        order : int
            Polynomial order (1 ~ 4 supported)
        """


        super().__init__()
        self.input_dim = input_dim
        self.order = order

        num_params = 0
        
        # --------------------------------------------------
        # 1st-order terms (linear effects)
        # --------------------------------------------------
        # z1, z2, ..., zd
        self.register_buffer("idx1", torch.arange(input_dim))
        num_params += input_dim


        # --------------------------------------------------
        # 2nd-order interaction terms
        # --------------------------------------------------
        # zi * zj for i < j
        if order >= 2:
            idx2_i, idx2_j = torch.triu_indices(input_dim, input_dim, offset=1)
            self.register_buffer("idx2_i", idx2_i)
            self.register_buffer("idx2_j", idx2_j)
            num_params += len(idx2_i)

        # --------------------------------------------------
        # 3rd-order interaction terms
        # --------------------------------------------------
        # zi * zj * zk
        if order >= 3:
            comb3 = torch.combinations(torch.arange(input_dim), r=3)
            self.register_buffer("idx3", comb3)
            num_params += comb3.shape[0]

        # --------------------------------------------------
        # 4th-order interaction terms
        # --------------------------------------------------
        # zi * zj * zk * zl

        if order >= 4:
            comb4 = torch.combinations(torch.arange(input_dim), r=4)
            self.register_buffer("idx4", comb4)
            num_params += comb4.shape[0]

        # --------------------------------------------------
        # Trainable coefficients β
        # --------------------------------------------------
        # Total parameters = Σ C(d, k)

        self.beta = torch.nn.Parameter(torch.zeros(num_params))



    def forward(self, X):
        """
        Compute Scheffé mean.

        Parameters
        ----------
        X : Tensor
            Shape (..., n, d)
            where d = input_dim

        Returns
        -------
        mean : Tensor
            Shape (..., n)
        """


        features = []

        # -------------------------
        # 1st order
        # -------------------------
        features.append(X)

        # -------------------------
        # 2nd order
        # -------------------------
        if self.order >= 2:
            F2 = X[..., self.idx2_i] * X[..., self.idx2_j]
            features.append(F2)

        # -------------------------
        # 3rd order
        # -------------------------
        if self.order >= 3:
            i, j, k = self.idx3.unbind(dim=1)
            F3 = X[..., i] * X[..., j] * X[..., k]
            features.append(F3)

        # -------------------------
        # 4th order
        # -------------------------
        if self.order >= 4:
            i, j, k, l = self.idx4.unbind(dim=1)
            F4 = X[..., i] * X[..., j] * X[..., k] * X[..., l]
            features.append(F4)

        # -------------------------
        # Concatenate all polynomial terms
        # -------------------------
        F = torch.cat(features, dim=-1)

        # Linear combination with β
        return torch.matmul(F, self.beta)


# ============================================================
# Monitoring Utilities
# ============================================================

logger = logging.getLogger("BO_Monitor")
logger.setLevel(logging.INFO)
if not logger.handlers:
    ch = logging.StreamHandler()
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    ch.setFormatter(formatter)
    logger.addHandler(ch)



def monitor(runtime=True, memory=True, condition=False):
    def decorator(func):
        @wraps(func)
        def wrapper(self, *args, **kwargs):
            if not getattr(self, "debug", False):
                return func(self, *args, **kwargs)
            if not hasattr(self, 'performance_history'):
                self.performance_history = []
            
            start_time = time.perf_counter()
            process = psutil.Process(os.getpid())
            
            result = func(self, *args, **kwargs)
            
            elapsed = time.perf_counter() - start_time
            cpu_mem = process.memory_info().rss / 1024**2
            
            entry = {
                "iteration": getattr(self, "current_iter", "N/A"),
                "function": func.__name__,
                "runtime_sec": round(elapsed, 4),
                "cpu_memory_mb": round(cpu_mem, 2),
                "timestamp": time.strftime("%Y-%m-%d %H:%M:%S")
            }
            if torch.cuda.is_available():
                entry["gpu_alloc_mb"] = round(torch.cuda.memory_allocated() / 1024**2, 2)
            
            self.performance_history.append(entry)
            logger.info(f"Finished {func.__name__} - {entry}")
            return result
        return wrapper
    return decorator



# ============================================================
# Scheffé Trend GP Emulator IDX ver.
# ============================================================
class IdxsScheffeTrendGPEmulator:
    """
    Universal Kriging model:

        f(x) = Scheffé polynomial trend + GP residual
    """

    def __init__(
        self,
        device        = torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        dtype         = torch.double,
        kernel        = 'rbf',
        use_ard       = False,
        matern_nu     = 2.5,
        scheffe_order = 2,
        debug         = False,
    ):
        self.device = device
        self.dtype = dtype
        self.kernel = kernel
        self.use_ard = use_ard
        self.matern_nu = matern_nu
        self.scheffe_order = scheffe_order
        self.debug = debug
        self.performance_history = []
        self.current_iter = 0
        self.model = None

    def _build_kernel(self, input_dim):
        ard_dims = input_dim if self.use_ard else None
        if self.kernel == "matern":
            base_kernel = MaternKernel(nu=self.matern_nu, ard_num_dims=ard_dims)
        else:
            base_kernel = RBFKernel(ard_num_dims=ard_dims)
        return ScaleKernel(base_kernel)

    @monitor(runtime=True, memory=True, condition=True)
    def fit(self, train_x: torch.Tensor, train_obj: torch.Tensor):
        """
        Fit multi-objective GP model.
        """
        train_x = train_x.to(device=self.device, dtype=self.dtype)
        train_obj = train_obj.to(device=self.device, dtype=self.dtype)

        if train_obj.ndim == 1:
            train_obj = train_obj.unsqueeze(-1)

        input_dim = train_x.shape[-1]
        num_objectives = train_obj.shape[-1]
        models = []

        for i in range(num_objectives):
            covar_module = self._build_kernel(input_dim)

            mean_module = IdxsFastScheffeMean(
                input_dim=input_dim,
                order=self.scheffe_order,
            ).to(device=self.device, dtype=self.dtype) 

            train_y = train_obj[..., i:i + 1]

            gp = SingleTaskGP(
                train_X=train_x,
                train_Y=train_y,
                mean_module=mean_module,
                outcome_transform=Standardize(m=1),
                covar_module=covar_module,
            )
            models.append(gp)

        self.model = ModelListGP(*models).to(device=self.device, dtype=self.dtype)
        self.mll = SumMarginalLogLikelihood(self.model.likelihood, self.model)

        with open(os.devnull, 'w') as f:
            with redirect_stdout(f):
                fit_gpytorch_mll(self.mll)

        return self.model

    def save_performance_to_json(self, folder_path, filename="performance_report.json"):
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
        full_path = os.path.join(folder_path, filename)
        try:
            with open(full_path, 'w', encoding='utf-8') as f:
                json.dump(self.performance_history, f, indent=4, ensure_ascii=False)
        except Exception as e:
            logger.error(f"Failed to save JSON: {str(e)}")

    @torch.no_grad()
    def predict(self, X):
        if self.model is None:
            raise RuntimeError("Model has not been fitted yet.")
        X = torch.as_tensor(X, dtype=self.dtype, device=self.device)
        posterior = self.model.posterior(X)
        return posterior.mean, posterior.variance

In [3]:
from botorch.models import KroneckerMultiTaskGP
from botorch.fit import fit_gpytorch_mll
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.models.transforms.outcome import Standardize


import logging
import json
import torch
import time
import psutil
import os
from functools import wraps
from contextlib import redirect_stdout
from botorch.fit import fit_gpytorch_mll
from botorch.models.transforms.outcome import Standardize


# --- Logging 配置 ---
logger = logging.getLogger("BO_Monitor")
logger.setLevel(logging.INFO)
if not logger.handlers:
    ch = logging.StreamHandler()
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    ch.setFormatter(formatter)
    logger.addHandler(ch)



# ============================================================
# 修復後的 Monitor Decorator
# ============================================================
def monitor(runtime=True, memory=True, condition=False):
    def decorator(func):
        @wraps(func)
        def wrapper(self, *args, **kwargs):
            if not getattr(self, "debug", False):
                return func(self, *args, **kwargs)
            
            if not hasattr(self, 'performance_history'):
                self.performance_history = []

            start_time = time.perf_counter()
            process = psutil.Process(os.getpid())
            
            result = func(self, *args, **kwargs)
            
            elapsed = time.perf_counter() - start_time
            cpu_mem = process.memory_info().rss / 1024**2
            
            entry = {
                "iteration": getattr(self, "current_iter", "N/A"),
                "function": func.__name__,
                "runtime_sec": round(elapsed, 4),
                "cpu_memory_mb": round(cpu_mem, 2),
                "timestamp": time.strftime("%Y-%m-%d %H:%M:%S")
            }

            if torch.cuda.is_available():
                entry["gpu_alloc_mb"] = round(torch.cuda.memory_allocated() / 1024**2, 2)

            # --- 修復後的條件數計算邏輯 ---
            if condition and getattr(self, "model", None) is not None:
                cond_list = []
                # 判斷是 ModelList 還是單一模型 (KMTGP/STGP)
                sub_models = self.model.models if hasattr(self.model, "models") else [self.model]
                
                for m in sub_models:
                    try:
                        with torch.no_grad():
                            # 獲取訓練協方差矩陣
                            # 注意：KMTGP 的矩陣可能非常大，這裡做 to_dense 會很吃記憶體
                            K = m.covar_module(m.train_inputs[0]).to_dense()
                            # 加上 jitter 提高穩定性
                            K += torch.eye(K.size(-1), device=K.device, dtype=K.dtype) * 1e-6
                            eigvals = torch.linalg.eigvalsh(K)
                            cond = (eigvals.max() / eigvals.min()).item()
                            cond_list.append(cond)
                    except Exception as e:
                        cond_list.append(float('nan'))
                entry["condition_numbers"] = cond_list

            self.performance_history.append(entry)
            logger.info(f"Finished {func.__name__} - {entry}")
            return result
        return wrapper
    return decorator



class CorrelationBaselineGPEmulator:
    """
    Correlation Baseline Multi-Objective GP Emulator
    =================================================

    This implementation uses KroneckerMultiTaskGP to jointly model
    multiple objectives while learning correlations between tasks.

    Mathematical Form:

        Y = f(X) + ε
        f ~ MultiTaskGP(m, K_x ⊗ K_t)

    where:

        K_x : covariance function over input space
        K_t : task covariance matrix modeling inter-task correlation

    Note:
        KroneckerMultiTaskGP requires all tasks to share
        the same training inputs (fully observed design).
    """


    def __init__(
        self,
        device  = torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        dtype   = torch.double,
        debug   = False,
    ):
        self.device = device
        self.dtype = dtype
        self.model = None
        self.mll = None
        self.debug = debug
        self.performance_history = []
        self.current_iter = 0



    @monitor(runtime=True, memory=True, condition=True)
    def fit(self, train_x: torch.Tensor, train_obj: torch.Tensor):
        """
        Fit a Kronecker Multi-Task GP model.

        Important:
            KroneckerMultiTaskGP assumes that all tasks are observed
            at the same set of training input locations.
        """

        # Convert single-objective case (N,) to (N, 1)
        if train_obj.ndim == 1:
            train_obj = train_obj.unsqueeze(-1)

        num_tasks = train_obj.shape[-1]

        # Construct the Kronecker Multi-Task GP
        # The model automatically learns the task covariance matrix
        self.model = KroneckerMultiTaskGP(
            train_X=train_x,
            train_Y=train_obj,
            outcome_transform=Standardize(m=num_tasks),
        )

        # Multi-task GP uses ExactMarginalLogLikelihood
        self.mll = ExactMarginalLogLikelihood(self.model.likelihood, self.model)

        # Train model while suppressing optimization output
        with open(os.devnull, 'w') as f:
            with redirect_stdout(f):
                fit_gpytorch_mll(self.mll)

        return self.model




    def save_performance_to_json(self, folder_path, filename="performance_report.json"):
        """
        將監控紀錄儲存至指定資料夾路徑。
        """
        # 1. 確保資料夾存在，若不存在則自動建立
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
            logger.info(f"Created directory: {folder_path}")

        # 2. 合併完整路徑
        full_path = os.path.join(folder_path, filename)

        # 3. 寫入 JSON
        try:
            with open(full_path, 'w', encoding='utf-8') as f:
                json.dump(self.performance_history, f, indent=4, ensure_ascii=False)
            logger.info(f"Successfully saved performance metrics to: {full_path}")
        except Exception as e:
            logger.error(f"Failed to save JSON: {str(e)}")



    @torch.no_grad()
    def predict(self, X: torch.Tensor):
        """
        Perform prediction using the trained model.

        Returns:
            posterior.mean      : shape (n, num_tasks)
            posterior.variance  : shape (n, num_tasks)
        """
        if self.model is None:
            raise RuntimeError("Model has not been fitted yet.")
        
        X = torch.as_tensor(X, dtype=self.dtype, device=self.device)
        
        # Switch to evaluation mode
        self.model.eval()
        self.model.likelihood.eval()
        
        posterior = self.model.posterior(X)
        
        # mean shape: (n, num_tasks)
        # variance shape: (n, num_tasks)
        return posterior.mean, posterior.variance

In [4]:
import torch
import itertools
import torch
import time
import psutil
import os
from functools import wraps
from contextlib import redirect_stdout
import logging
import json


from botorch.fit import fit_gpytorch_mll
from botorch.models.transforms.outcome import Standardize
from gpytorch.means import Mean
from botorch.models import KroneckerMultiTaskGP
from gpytorch.mlls import ExactMarginalLogLikelihood



# ============================================================
# Optimized Mean Function: IDXSFastScheffeMean 
# ============================================================
class IdxsFastScheffeMean(Mean):
    """
    High-performance Scheffé polynomial mean function.

    Mathematical form:

        m(z) =
            Σ β_i z_i
          + Σ β_ij z_i z_j
          + Σ β_ijk z_i z_j z_k
          + Σ β_ijkl z_i z_j z_k z_l

    depending on order.

    Design goals:
    - No torch.pow
    - No mask matrix
    - No large broadcast tensor
    - No Python inner loops in forward
    - Optimized for optimize_acqf gradient loops
    """

    def __init__(self, input_dim, order=2):

        """
        Parameters
        ----------
        input_dim : int
            Number of mixture components (dimension d)

        order : int
            Polynomial order (1 ~ 4 supported)
        """


        super().__init__()
        self.input_dim = input_dim
        self.order = order

        num_params = 0
        
        # --------------------------------------------------
        # 1st-order terms (linear effects)
        # --------------------------------------------------
        # z1, z2, ..., zd
        self.register_buffer("idx1", torch.arange(input_dim))
        num_params += input_dim


        # --------------------------------------------------
        # 2nd-order interaction terms
        # --------------------------------------------------
        # zi * zj for i < j
        if order >= 2:
            idx2_i, idx2_j = torch.triu_indices(input_dim, input_dim, offset=1)
            self.register_buffer("idx2_i", idx2_i)
            self.register_buffer("idx2_j", idx2_j)
            num_params += len(idx2_i)

        # --------------------------------------------------
        # 3rd-order interaction terms
        # --------------------------------------------------
        # zi * zj * zk
        if order >= 3:
            comb3 = torch.combinations(torch.arange(input_dim), r=3)
            self.register_buffer("idx3", comb3)
            num_params += comb3.shape[0]

        # --------------------------------------------------
        # 4th-order interaction terms
        # --------------------------------------------------
        # zi * zj * zk * zl

        if order >= 4:
            comb4 = torch.combinations(torch.arange(input_dim), r=4)
            self.register_buffer("idx4", comb4)
            num_params += comb4.shape[0]

        # --------------------------------------------------
        # Trainable coefficients β
        # --------------------------------------------------
        # Total parameters = Σ C(d, k)

        self.beta = torch.nn.Parameter(torch.zeros(num_params))



    def forward(self, X):
        """
        Compute Scheffé mean.

        Parameters
        ----------
        X : Tensor
            Shape (..., n, d)
            where d = input_dim

        Returns
        -------
        mean : Tensor
            Shape (..., n)
        """


        features = []

        # -------------------------
        # 1st order
        # -------------------------
        features.append(X)

        # -------------------------
        # 2nd order
        # -------------------------
        if self.order >= 2:
            F2 = X[..., self.idx2_i] * X[..., self.idx2_j]
            features.append(F2)

        # -------------------------
        # 3rd order
        # -------------------------
        if self.order >= 3:
            i, j, k = self.idx3.unbind(dim=1)
            F3 = X[..., i] * X[..., j] * X[..., k]
            features.append(F3)

        # -------------------------
        # 4th order
        # -------------------------
        if self.order >= 4:
            i, j, k, l = self.idx4.unbind(dim=1)
            F4 = X[..., i] * X[..., j] * X[..., k] * X[..., l]
            features.append(F4)

        # -------------------------
        # Concatenate all polynomial terms
        # -------------------------
        F = torch.cat(features, dim=-1)

        # Linear combination with β
        return torch.matmul(F, self.beta)
    

# ============================================================
# Monitoring Utilities & Decorator
# ============================================================
logger = logging.getLogger("BO_Monitor")
logger.setLevel(logging.INFO)
if not logger.handlers:
    ch = logging.StreamHandler()
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    ch.setFormatter(formatter)
    logger.addHandler(ch)




# ============================================================
# Monitor Decorator
# ============================================================
def monitor(runtime=True, memory=True, condition=False):
    def decorator(func):
        @wraps(func)
        def wrapper(self, *args, **kwargs):
            if not getattr(self, "debug", False):
                return func(self, *args, **kwargs)
            
            if not hasattr(self, 'performance_history'):
                self.performance_history = []

            start_time = time.perf_counter()
            process = psutil.Process(os.getpid())
            
            result = func(self, *args, **kwargs)
            
            elapsed = time.perf_counter() - start_time
            cpu_mem = process.memory_info().rss / 1024**2
            
            entry = {
                "iteration": getattr(self, "current_iter", "N/A"),
                "function": func.__name__,
                "runtime_sec": round(elapsed, 4),
                "cpu_memory_mb": round(cpu_mem, 2),
                "timestamp": time.strftime("%Y-%m-%d %H:%M:%S")
            }

            if torch.cuda.is_available():
                entry["gpu_alloc_mb"] = round(torch.cuda.memory_allocated() / 1024**2, 2)

            # --- 修復後的條件數計算邏輯 ---
            if condition and getattr(self, "model", None) is not None:
                cond_list = []
                # 判斷是 ModelList 還是單一模型 (KMTGP/STGP)
                sub_models = self.model.models if hasattr(self.model, "models") else [self.model]
                
                for m in sub_models:
                    try:
                        with torch.no_grad():
                            # 獲取訓練協方差矩陣
                            # 注意：KMTGP 的矩陣可能非常大，這裡做 to_dense 會很吃記憶體
                            K = m.covar_module(m.train_inputs[0]).to_dense()
                            # 加上 jitter 提高穩定性
                            K += torch.eye(K.size(-1), device=K.device, dtype=K.dtype) * 1e-6
                            eigvals = torch.linalg.eigvalsh(K)
                            cond = (eigvals.max() / eigvals.min()).item()
                            cond_list.append(cond)
                    except Exception as e:
                        cond_list.append(float('nan'))
                entry["condition_numbers"] = cond_list

            self.performance_history.append(entry)
            logger.info(f"Finished {func.__name__} - {entry}")
            return result
        return wrapper
    return decorator




class CorrelationIdxsScheffeTrendGPEmulator:
    """
    Correlated Multi-Objective Gaussian Process Emulator
    ====================================================

    This class builds a multi-objective GP surrogate model:

        y_k(z) = m_k(z) + g_k(z)

    where:
        m_k(z) = Scheffé polynomial trend
        g_k(z) ~ GP(0, K_θ)

    The correlation between objectives is modeled via
    KroneckerMultiTaskGP.

    Parameters
    ----------
    device : torch.device
        Device for computation (CPU or GPU).

    dtype : torch.dtype
        Floating point precision.

    scheffe_order : int
        Polynomial order of Scheffé mean.

    debug : bool
        Enable runtime/memory diagnostics.
    """
    def __init__(
        self,
        device        = torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        dtype         = torch.double,
        scheffe_order = 2,
        debug         = False,   # Debug switch

    ):
        self.device = device
        self.dtype = dtype
        self.model = None
        self.mll = None

        self.scheffe_order = scheffe_order
        self.debug = debug

        self.performance_history = []
        self.current_iter = 0
        self.model = None


    @monitor(runtime=True, memory=True, condition=False)
    def fit(self, train_x: torch.Tensor, train_obj: torch.Tensor):
        """
        Fit the multi-task GP model.

        Parameters
        ----------
        train_x : torch.Tensor
            Shape (N, d)

        train_obj : torch.Tensor
            Shape (N, k)
            k = number of objectives
        """

        # If single objective (N,), convert to (N, 1)
        if train_obj.ndim == 1:
            train_obj = train_obj.unsqueeze(-1)

        input_dim = train_x.shape[-1]
        num_objectives = train_obj.shape[-1]


        # Define Scheffé polynomial mean
        mean_module = IdxsFastScheffeMean(
            input_dim=input_dim,
            order=self.scheffe_order,
        )  


        # Construct correlated multi-task GP
        self.model = KroneckerMultiTaskGP(
            train_X=train_x,
            train_Y=train_obj,
            outcome_transform=Standardize(m=num_objectives),
            mean_module = mean_module
        )

        # Exact marginal log likelihood
        self.mll = ExactMarginalLogLikelihood(self.model.likelihood, self.model)

        with open(os.devnull, 'w') as f:
            with redirect_stdout(f):
                fit_gpytorch_mll(self.mll)

        return self.model
    
    def save_performance_to_json(self, folder_path, filename="performance_report.json"):
        """
        將監控紀錄儲存至指定資料夾路徑。
        """
        # 1. 確保資料夾存在，若不存在則自動建立
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
            logger.info(f"Created directory: {folder_path}")

        # 2. 合併完整路徑
        full_path = os.path.join(folder_path, filename)

        # 3. 寫入 JSON
        try:
            with open(full_path, 'w', encoding='utf-8') as f:
                json.dump(self.performance_history, f, indent=4, ensure_ascii=False)
            logger.info(f"Successfully saved performance metrics to: {full_path}")
        except Exception as e:
            logger.error(f"Failed to save JSON: {str(e)}")

            
    
    @torch.no_grad()
    def predict(self, X ):
        """
        Posterior prediction.

        Parameters
        ----------
        X : torch.Tensor or array-like
            Shape (n, d)

        Returns
        -------
        mean : torch.Tensor
            Posterior predictive mean, shape (n, k)

        var : torch.Tensor
            Posterior predictive variance, shape (n, k)
        """

        if self.model is None:
            raise RuntimeError("Model has not been fitted yet.")
        
        X = torch.as_tensor(X, dtype=self.dtype, device=self.device)
        posterior = self.model.posterior(X)
        mean = posterior.mean    # (n, k)
        var = posterior.variance # (n, k)

        return mean, var


In [5]:

import torch
import pandas as pd
import os
import numpy as np
import json
from botorch.models import SingleTaskGP, ModelListGP
from botorch.models.transforms import Standardize, Normalize
from gpytorch.mlls.sum_marginal_log_likelihood import SumMarginalLogLikelihood
from botorch.sampling.normal import SobolQMCNormalSampler
from botorch.acquisition.multi_objective.logei import (
    qLogExpectedHypervolumeImprovement,
    qLogNoisyExpectedHypervolumeImprovement
)
from botorch import fit_gpytorch_mll
from gpytorch.mlls import ExactMarginalLogLikelihood




# 設定設備與型別
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float64
torch.set_default_dtype(dtype)
print('==='*5)
print('device = ',device)
print('==='*5)

FEATURE_COLS = [
    'AA001','AA002','AA004','AA005','AA006','AW001','AW003','AW004','AW005','AW014',
    'AX010','AX015','AX019','AX020','AX029','AX032','AX137','CM1002','CM1007','CM1008',
    'CP2002','FR001','FR002','FR006','FR007','FR008','FR015','FR058','GF001','GF006',
    'GF013','GF014','GF016','GF020','MF001','MF005','MF006','MF007','PR002','PR007',
    'PR009','PR016','PR020','PR022','PR024','SS004','SS010'
    ]
TARGET_COLS = ["SPGR", "TE"]


# =====================================================
# Oracle loader (standalone, aligned with colleague)
# =====================================================
def load_oracle(beta_csv_path):
    df = pd.read_csv(beta_csv_path)

    if "active" in df.columns:
        df = df[df["active"].astype(bool)].copy()

    # intercept
    inter = df[df["type"].str.lower() == "intercept_correction"]
    intercept = torch.tensor(
        inter.iloc[0][TARGET_COLS].astype(float).to_numpy(),
        device=device,
        dtype=dtype,
    )

    feat_to_idx = {f: i for i, f in enumerate(FEATURE_COLS)}

    # linear
    lin_df = df[df["type"].str.lower() == "linear"].copy()
    lin_df["feat"] = lin_df["feature"].str.extract(r"x\[(.+?)\]")
    lin_df["idx"] = lin_df["feat"].map(feat_to_idx)

    beta_lin = torch.zeros((2, len(FEATURE_COLS)), device=device, dtype=dtype)
    for _, r in lin_df.iterrows():
        beta_lin[:, int(r["idx"])] = torch.tensor(
            r[TARGET_COLS].astype(float).to_numpy(),
            device=device,
            dtype=dtype,
        )

    # interaction
    int_df = df[df["type"].str.lower() == "interaction"].copy()
    mats = int_df["feature"].str.extract(r"x\[(.+?)\]\*x\[(.+?)\]")
    int_df["i"] = mats[0].map(feat_to_idx)
    int_df["j"] = mats[1].map(feat_to_idx)

    pairs = torch.tensor(
        int_df[["i", "j"]].values.astype(int),
        device=device,
        dtype=torch.long,
    )

    beta_inter = torch.tensor(
        int_df[TARGET_COLS].astype(float).to_numpy(),
        device=device,
        dtype=dtype,
    ).T

    return {
        "intercept": intercept,
        "beta_lin": beta_lin,
        "pairs": pairs,
        "beta_inter": beta_inter,
    }



def oracle_function(X, oracle):
    intercept = oracle["intercept"]
    beta_lin = oracle["beta_lin"]
    pairs = oracle["pairs"]
    beta_inter = oracle["beta_inter"]

    lin = X @ beta_lin.T
    cross = X[:, pairs[:, 0]] * X[:, pairs[:, 1]]
    inter = cross @ beta_inter.T
    return intercept.unsqueeze(0) + lin + inter

device =  cuda


In [6]:
from botorch.optim import optimize_acqf
from botorch.utils.multi_objective.pareto import is_non_dominated
from botorch.utils.multi_objective.hypervolume import Hypervolume


seed  = '39'
PATH  = "D:/Users/TingYuLin/Desktop/py12/MOBO/synthetic_data_sparse_constraint_seed_39.csv"
# PATH2 = f"./synthetic_data_sparse_seed_{seed}.csv"
N_ITER = 50
df = pd.read_csv(PATH)

train_x = torch.tensor(df[FEATURE_COLS].values/100, device=device, dtype=dtype)
train_obj = torch.tensor(df[TARGET_COLS].values, device=device, dtype=dtype)

hv_history = []
pareto_history = []
all_X = [train_x.detach().cpu().numpy().tolist()]  # 存初始 X
all_Y = [train_obj.detach().cpu().numpy().tolist()]  # 存初始 Y


print(train_obj.shape)
ref_point = torch.tensor([1.3654, 2.8482], dtype=dtype, device=device)



SAVE_DIR = "D:/Users/TingYuLin/Desktop/py12/MOBO/Scheffe_gp/base_gp_result"
# SAVE_DIR = "D:/Users/TingYuLin/Desktop/py12/MOBO/Scheffe_gp/base_schyeffe_result"
# SAVE_DIR = "D:/Users/TingYuLin/Desktop/py12/MOBO/Scheffe_gp/coor_gp_result"
# SAVE_DIR = "D:/Users/TingYuLin/Desktop/py12/MOBO/Scheffe_gp/coor_base_schyeffe_result"

FILE_NAME = f"final_performance_seed_{seed}.json"


BETA_CSV = "D:/Users/TingYuLin/Desktop/py12/MOBO/beta1.csv"
oracle = load_oracle(BETA_CSV)


gp_base = BaselineGPEmulator(kernel = 'rbf',debug=True)
# gp_Scheffe = IdxsScheffeTrendGPEmulator(kernel = 'rbf', scheffe_order = 2, debug=True)
# corr_gp_base = CorrelationBaselineGPEmulator(debug=True)
# corr_Scheffe = CorrelationIdxsScheffeTrendGPEmulator(scheffe_order = 2, debug=True)


input_dim = train_x.shape[-1]
for it in range(N_ITER):


    gp_base.current_iter = it
    # gp_Scheffe.current_iter = it
    # corr_gp_base.current_iter = it
    # corr_Scheffe.current_iter = it


    start_time = time.perf_counter()
    model = gp_base.fit(train_x,train_obj)
    # model = gp_Scheffe.fit(train_x,train_obj)
    # model = corr_gp_base.fit(train_x,train_obj)
    # model = corr_Scheffe.fit(train_x,train_obj)

    end_time = time.perf_counter()
    elapsed = end_time - start_time

    print('---'*10)
    print('model fit time  = ', elapsed)

    if it % 5 == 0:
        gp_base.save_performance_to_json(SAVE_DIR, f"perf_iter_{it}.json")
        # gp_Scheffe.save_performance_to_json(SAVE_DIR, f"perf_iter_{it}.json")
        # corr_gp_base.save_performance_to_json(SAVE_DIR, f"perf_iter_{it}.json")
        # corr_Scheffe.save_performance_to_json(SAVE_DIR, f"perf_iter_{it}.json")

    start_time1 = time.perf_counter()

    sampler = SobolQMCNormalSampler(sample_shape=torch.Size([128]))
    
    acq_func = qLogNoisyExpectedHypervolumeImprovement(
            model=model,
            ref_point=ref_point,
            X_baseline=train_x,
            sampler=sampler,
            prune_baseline=True,
        )


    end_time1 = time.perf_counter()
    elapsed1 = end_time1 - start_time1


    print('qLogNoisyExpectedHypervolumeImprovement time = ', elapsed1)

    bounds=torch.stack([
        torch.zeros(len(FEATURE_COLS), device=device),
        torch.ones(len(FEATURE_COLS), device=device),
    ])

    equality_constraints=[
        (
            torch.arange(len(FEATURE_COLS), device=device),
            torch.ones(len(FEATURE_COLS), device=device),
            1.0,
        )
    ]


    start_time2 = time.perf_counter()
    new_x, _ = optimize_acqf(
        acq_function = acq_func, 
        bounds = bounds, 
        q= 1, 
        num_restarts = 10, 
        raw_samples = 128, 
        equality_constraints = [
            (
                torch.arange(len(FEATURE_COLS), device=device),
                torch.ones(len(FEATURE_COLS), device=device),
                1.0,
            )
        ],
        options={
            "batch_limit": 5, 
            "maxiter": 200
        }
    )

    end_time2 = time.perf_counter()
    elapsed2 = end_time2 - start_time2

    print('optimize_acqf time = ', elapsed2)

    # pre_mean,pre_var = corr_Scheffe.predict(new_x)

    # print('--'*50)
    # print('pre_mean = ',pre_mean)
    # print('pre_var = ',pre_var)
    # print('--'*50)

    start_time3 = time.perf_counter()
    new_y = oracle_function(new_x, oracle)  # shape (1, 2)

    # ============================
    # Append new data
    # ============================
    train_x = torch.cat([train_x, new_x], dim=0)
    train_obj = torch.cat([train_obj, new_y], dim=0)

        # 存 X, Y
    all_X.append(new_x.detach().cpu().numpy().tolist())
    all_Y.append(new_y.detach().cpu().numpy().tolist())

        # ============================
    # Compute HV
    # ============================
    nd_mask = is_non_dominated(train_obj)
    pareto = train_obj[nd_mask]

    hv = Hypervolume(ref_point)
    hv_val = hv.compute(pareto)

    hv_history.append(hv_val)
    pareto_history.append(pareto.detach().cpu().tolist())


    end_time3 = time.perf_counter()
    elapsed3 = end_time3 - start_time3
    print('Append new data time = ', elapsed3)
    print('---'*10)



    print(f"Iter {it:02d} | HV = {hv_val:.4f}")


# =================================================
# Save JSON per seed
# =================================================
out = {
    "seed": seed,
    "hv_history": hv_history,
    "pareto_history": pareto_history,
    "all_X": all_X,
    "all_Y": all_Y,
    "n_evals": train_x.shape[0],
}


with open(f"{SAVE_DIR}/synthetic_data_sparse_seed_{seed}.json", "w") as f:
    json.dump(out, f, indent=2)

print(f"Saved → {SAVE_DIR}/synthetic_data_sparse_seed_{seed}.json")

# 最終存檔
gp_base.save_performance_to_json(SAVE_DIR, FILE_NAME)
# gp_Scheffe.save_performance_to_json(SAVE_DIR, FILE_NAME)
# corr_gp_base.save_performance_to_json(SAVE_DIR, FILE_NAME)
# corr_Scheffe.save_performance_to_json(SAVE_DIR, FILE_NAME)



torch.Size([100, 2])


2026-02-26 09:17:21,613 - INFO - Finished fit - {'iteration': 0, 'function': 'fit', 'runtime_sec': 1.5435, 'cpu_memory_mb': 968.36, 'timestamp': '2026-02-26 09:17:21', 'gpu_alloc_mb': 16.34, 'condition_numbers': [1550202.2893797294, 216063.2665307227]}
2026-02-26 09:17:21,613 - INFO - Created directory: D:/Users/TingYuLin/Desktop/py12/MOBO/Scheffe_gp/base_gp_result
2026-02-26 09:17:21,620 - INFO - Successfully saved performance metrics to: D:/Users/TingYuLin/Desktop/py12/MOBO/Scheffe_gp/base_gp_result\perf_iter_0.json


------------------------------
model fit time  =  1.5536684999242425
qLogNoisyExpectedHypervolumeImprovement time =  0.1468301999848336
optimize_acqf time =  2.077849799999967
Append new data time =  0.0022368000354617834
------------------------------
Iter 00 | HV = 0.0047


2026-02-26 09:17:24,251 - INFO - Finished fit - {'iteration': 1, 'function': 'fit', 'runtime_sec': 0.3965, 'cpu_memory_mb': 1250.44, 'timestamp': '2026-02-26 09:17:24', 'gpu_alloc_mb': 17.04, 'condition_numbers': [1765742.4494488677, 226798.01829249453]}


------------------------------
model fit time  =  0.40388659993186593
qLogNoisyExpectedHypervolumeImprovement time =  0.07702270010486245
optimize_acqf time =  2.510609900113195
Append new data time =  0.0022243999410420656
------------------------------
Iter 01 | HV = 0.0054


...

------------------------------
model fit time  =  0.4751167001668364
qLogNoisyExpectedHypervolumeImprovement time =  0.11642810003831983
optimize_acqf time =  11.966160699957982
Append new data time =  0.02590239979326725
------------------------------
Iter 48 | HV = 0.0073


2026-02-26 09:22:57,297 - INFO - Finished fit - {'iteration': 49, 'function': 'fit', 'runtime_sec': 0.55, 'cpu_memory_mb': 1307.16, 'timestamp': '2026-02-26 09:22:57', 'gpu_alloc_mb': 41.15, 'condition_numbers': [-3.783011208471727e+17, -3214406733503738.5]}
d:\Users\TingYuLin\Desktop\py12\venv\Lib\site-packages\linear_operator\utils\cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(


------------------------------
model fit time  =  0.5670469999313354
qLogNoisyExpectedHypervolumeImprovement time =  0.13670890009962022


2026-02-26 09:23:03,205 - INFO - Successfully saved performance metrics to: D:/Users/TingYuLin/Desktop/py12/MOBO/Scheffe_gp/base_gp_result\final_performance_seed_39.json


optimize_acqf time =  5.735966400010511
Append new data time =  0.018379800021648407
------------------------------
Iter 49 | HV = 0.0073
Saved → D:/Users/TingYuLin/Desktop/py12/MOBO/Scheffe_gp/base_gp_result/synthetic_data_sparse_seed_39.json


In [7]:
# import json
# import matplotlib.pyplot as plt

# def plot_averages(file_path):
#     with open(file_path, 'r', encoding='utf-8') as f:
#         data = json.load(f)
#     if not data:
#         return "JSON 文件中沒有數據。"
    
#     hv_history = data.get('hv_history')

#     return hv_history


# def tree_averages(file_path):
#     with open(file_path, 'r', encoding='utf-8') as f:
#         data = json.load(f)
#     if not data:
#         return "JSON 文件中沒有數據。"
    
#     return data



# base_gp_hv_history = plot_averages("D:/Users/TingYuLin/Desktop/py12/MOBO/Scheffe_gp/base_gp_result/synthetic_data_sparse_seed_39.json")
# Scheffé_gp_hv_history = plot_averages("D:/Users/TingYuLin/Desktop/py12/MOBO/Scheffe_gp/base_schyeffe_result/synthetic_data_sparse_seed_39.json")
# rf_history = tree_averages("D:/Users/TingYuLin/Desktop/py12/MOBO/Scheffe_gp/tree_result/hv_history.json")
# gb_history = tree_averages("D:/Users/TingYuLin/Desktop/py12/MOBO/Scheffe_gp/tree_result/hv_history.json")


# # 3. 繪圖
# plt.figure(figsize=(10, 6))
# plt.plot(base_gp_hv_history, marker='o', linestyle='-', color='#1f77b4', markersize=4, label='without Scheffé HV')
# plt.plot(Scheffé_gp_hv_history, marker='o', linestyle='-', color="#49b41f", markersize=4, label='with Scheffé HV')
# plt.plot(rf_history, marker='o', linestyle='-', color="#c51515", markersize=4, label='Random forest HV')
# plt.plot(gb_history, marker='o', linestyle='-', color="#000000", markersize=4, label=' Gradient Boosting HV')


# # # 計算累積最大值 (Running Maximum) 以顯示收斂狀況
# # running_max = [max(hv_history[:i+1]) for i in range(len(hv_history))]
# # plt.step(range(len(running_max)), running_max, where='post', color='red', alpha=0.6, label='Best HV')

# # 圖表格式設定
# plt.title('Hypervolume (HV) Convergence - Multi-Objective BO', fontsize=14)
# plt.xlabel('Iteration (Batch)', fontsize=12)
# plt.ylabel('Hypervolume Indicator', fontsize=12)
# plt.grid(True, linestyle='--', alpha=0.7)
# plt.legend()

# plt.tight_layout()
# plt.show()


In [8]:
# import json

# # 假設您的檔案名稱為 performance_report.json
# def calculate_performance_averages(file_path):
#     with open(file_path, 'r', encoding='utf-8') as f:
#         data = json.load(f)

#     if not data:
#         return "JSON 文件中沒有數據。"

#     n = len(data)
    
#     # 提取各項數值
#     runtimes = [entry['runtime_sec'] for entry in data]
#     cpu_mems = [entry['cpu_memory_mb'] for entry in data]
#     gpu_mems = [entry.get('gpu_alloc_mb', 0) for entry in data] # 使用 get 以防某些紀錄沒用到 GPU

#     # 計算平均
#     avg_runtime = sum(runtimes) / n
#     avg_cpu = sum(cpu_mems) / n
#     avg_gpu = sum(gpu_mems) / n

#     print(f"--- Scheffe mean 統計報告 (共 {n} 筆紀錄) ---")
#     print(f"平均執行時間: {avg_runtime:.4f} sec")
#     print(f"平均 CPU 使用量: {avg_cpu:.2f} MB")
#     print(f"平均 GPU 佔用量: {avg_gpu:.2f} MB")

# # 執行範例
# calculate_performance_averages("D:/Users/TingYuLin/Desktop/py12/MOBO/Scheffe_gp/base_schyeffe_result/final_performance_seed_39.json")